## Calculate the volume integral of a given 3D permeability field using SYCL parallel computing

The problem statement solved by the given code is to calculate the volume integral of a given 3D permeability field using SYCL parallel computing.

The code first initializes a 3D permeability field of size NX x NY x NZ with values 1, 2, 3, ..., NXNYNZ using the std::iota() function. Then, it creates SYCL buffers for the permeability field and a result variable.

Next, the code submits a SYCL kernel using a parallel_for loop that calculates the volume integral of the permeability field. The kernel multiplies all the values in the permeability field together and stores the result in the result variable.

The code runs the kernel on a chosen device, either a CPU or GPU, depending on the selector that is uncommented. The device name is printed to the console to confirm which device is being used. After the kernel completes, the result of the volume integral is printed to the console.

Note that the volume integral of a 3D field is calculated by taking the product of all the values in the field. In this case, the permeability field represents the permeability of a porous medium, and the volume integral is a measure of the overall permeability of the medium.







In [1]:
%%writefile lab/simple1.cpp
#include <CL/sycl.hpp>
#include <iostream>
#include <numeric>

using namespace cl::sycl;

int main(int argc, char** argv) {

  constexpr size_t NX = 10;
  constexpr size_t NY = 10;
  constexpr size_t NZ = 10;

  std::vector<float> perm(NX*NY*NZ);
  std::iota(perm.begin(), perm.end(), 1.0f); // populate with values 1, 2, 3, ..., NX*NY*NZ

  float result = 1.0f;

  try {
    //queue q{gpu_selector_v<cl::sycl::gpu_selector>{}};
    //queue q{sycl::gpu_selector_v{}};
     // queue q{sycl::gpu_selector_v<cl::sycl::gpu_selector>{}};
      queue q{sycl::cpu_selector{}};
    std::cout << "Running on "
              << q.get_device().get_info<info::device::name>()
              << std::endl;

    buffer<float, 1> perm_buf(perm.data(), range<1>(NX*NY*NZ));
    buffer<float, 1> result_buf(&result, range<1>(1));

    q.submit([&](handler& cgh) {
      auto perm_acc = perm_buf.get_access<access::mode::read>(cgh);
      auto result_acc = result_buf.get_access<access::mode::write>(cgh);

      cgh.parallel_for<class perm_volume_integral>(
        range<1>(NX*NY*NZ),
        [=](id<1> idx) {
          result_acc[0] *= perm_acc[idx];
        }
      );
    });

    std::cout << "Perm volume integral: " << result << std::endl;

  } catch (sycl::exception& e) {
    std::cerr << "SYCL exception caught: " << e.what() << std::endl;
    return 1;
  }

  return 0;
}

Overwriting lab/simple1.cpp


In [2]:
! chmod 755 q; chmod 755 run_simple1.sh;if [ -x "$(command -v qsub)" ]; then ./q run_simple1.sh; else ./run_simple1.sh; fi

Job has been submitted to Intel(R) DevCloud and will execute soon.

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2260557.v-qsvr-1           ...ub-singleuser u182761         00:00:10 R jupyterhub     
2260574.v-qsvr-1           run_simple.sh    u182761         00:08:02 R batch          
2260582.v-qsvr-1           run_simple1.sh   u182761                0 Q batch          

Waiting for Output █████████████████████ Done⬇

########################################################################
#      Date:           Wed 22 Mar 2023 11:49:29 PM PDT
#    Job ID:           2260582.v-qsvr-1.aidevcloud
#      User:           u182761
# Resources:           cput=75:00:00,neednodes=1:gpu:ppn=2,nodes=1:gpu:ppn=2,walltime=06:00:00
########################################################################

## u182761 is compiling DPCPP_Essentials Module1 -- oneAPI Techgig example - 1 of 1 simple1